<br><br><font color="gray">DOING COMPUTATIONAL SOCIAL SCIENCE<br>MODULE 8 <strong>PROBLEM SETS</strong></font>

# <font color="#49699E" size=40>MODULE 8 </font>


# What You Need to Know Before Getting Started

- **Every notebook assignment has an accompanying quiz**. Your work in each notebook assignment will serve as the basis for your quiz answers.
- **You can consult any resources you want when completing these exercises and problems**. Just as it is in the "real world:" if you can't figure out how to do something, look it up. My recommendation is that you check the relevant parts of the assigned reading or search for inspiration on [https://stackoverflow.com](https://stackoverflow.com).
- **Each problem is worth 1 point**. All problems are equally weighted.
- **The information you need for each problem set is provided in the blue and green cells.** General instructions / the problem set preamble are in the blue cells, and instructions for specific problems are in the green cells. **You have to execute all of the code in the problem set, but you are only responsible for entering code into the code cells that immediately follow a green cell**. You will also recognize those cells because they will be incomplete. You need to replace each blank `▰▰#▰▰` with the code that will make the cell execute properly (where # is a sequentially-increasing integer, one for each blank).
- Most modules will contain at least one question that requires you to load data from disk; **it is up to you to locate the data, place it in an appropriate directory on your local machine, and replace any instances of the `PATH_TO_DATA` variable with a path to the directory containing the relevant data**.
- **The comments in the problem cells contain clues indicating what the following line of code is supposed to do.** Use these comments as a guide when filling in the blanks. 
- **You can ask for help**. 

Finally, remember that you do not need to "master" this content before moving on to other course materials, as what is introduced here is reinforced throughout the rest of the course. You will have plenty of time to practice and cement your new knowledge and skills.
<div class='alert alert-block alert-danger'>As you complete this assignment, you may encounter variables that can be assigned a wide variety of different names. Rather than forcing you to employ a particular convention, we leave the naming of these variables up to you. During the quiz, submit an answer of 'USER_DEFINED' (without the quotation marks) to fill in any blank that you assigned an arbitrary name to. In most circumstances, this will occur due to the presence of a local iterator in a for-loop.</b></div>

## Package Imports

In [1]:
import pandas as pd
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import random

import numpy as np

from dcss.networks import *

import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.utils import multi_runs

%config Completer. use_jedi = False

def simulation_overview(iteration_results, network, prop=True):
    population_size = network.number_of_nodes()

    trends = []
    deltas = []

    for iteration in iteration_results:
        trends.append(iteration['node_count'])
        deltas.append(iteration['status_delta'])

    columns = ['Susceptible', 'Infected', 'Removed']

    # trends DF
    trends = pd.DataFrame(trends)
    trends.columns = columns
    if prop is True:
        trends = trends.div(population_size)

    # deltas DF
    deltas = pd.DataFrame(deltas)
    deltas.columns = columns

    return trends, deltas


def sir_model(network, beta, gamma, fraction_infected):
    model = ep.SIRModel(network)

    config = mc.Configuration()
    config.add_model_parameter('beta', beta)
    config.add_model_parameter('gamma', gamma)
    config.add_model_parameter("fraction_infected", fraction_infected)
    
    model.set_initial_status(config)
    return model 

## Problem 1:
<div class="alert alert-block alert-info">  
For this assignment, we're going explore how simple contagions might spread through small networks of face-to-face contact. We'll start by generating a Watts-Strogratz Graph that will form the basis of the remainder of our investigation.
</div>
<div class="alert alert-block alert-success">
Using NetworkX, create a Watts-Strogratz Graph with p=0.15, n=500, and k=4. Instead of using keyword arguments, input your arguments as positional parameters. If you need some assistance as to which argument should go where, you can view the function's documentation by running the <code>?nx.watts_strogatz_graph</code> in a separate cell or consulting the NetworkX documentation directly. (The <code>?</code> command is very handy: it can be used to retrieve a docstring from any Python object)
</div>

In [ ]:
G = nx.▰▰1▰▰(
    ▰▰2▰▰, 
    ▰▰3▰▰, 
    ▰▰4▰▰, 
    seed=42)

pos = nx.nx_pydot.graphviz_layout(G)

fig, ax = plt.subplots(figsize=(12, 12))
nx.draw(
    G, 
    pos,
    ax=ax,
    node_color="grey",
    edge_color="grey",
    node_size=100,
    width=.5
)
plt.show()

## Problem 2:
<div class="alert alert-block alert-info">  
In the previous assignment, we devoted a significant amount of time to exploring how four different measures of centrality (shortest-path, current flow, degree, and eigenvector) differ from one another. Intuitively, one might surmise that highly central nodes are more likely to spread simple contagions through a network more rapidly than less-central nodes. Thinking about this might cause one to wonder: "Which of the various forms of centrality is the best predictor of a node's impact on diffusion processes?"<br><br>
In an attempt to provide a (limited, provisional) answer to this question, we'll use the centrality measures from last week: you can produce dictionaries of them by re-using some of your code from the previous assignment!
</div>
<div class="alert alert-block alert-success">
Create four centrality dictionaries (one each for shortest-path, current flow, degree, and eigenvector) for your Watts-Strogatz graph.
</div>

In [ ]:
# Create dictionary of shortest-path betweenness centrality
shortest_dict = nx.▰▰1▰▰(▰▰2▰▰)

# Create dictionary of current-flow betweenness centrality
current_dict = nx.▰▰3▰▰(▰▰4▰▰)

# Create a dictionary of degree centralities
deg_cent_dict = nx.▰▰5▰▰(▰▰6▰▰)

# Create a dictionary of eigenvector centralities
eigen_dict = nx.▰▰7▰▰(▰▰8▰▰, max_iter=1000)


## Problem 3:
<div class="alert alert-block alert-info">  
While having all of the centralities available can be useful, we're only interested in a small subset of the nodes that represent those in the network with the highest or lowest centrality scores. Rather than individually processing each list of the node centralities (which was what we did last week), it'll be more efficient to write a function that can handle multiple centrality dictionaries at the same time.
</div>
<div class="alert alert-block alert-success">
Finish writing this function that retrieves the `k` most or least central nodes from each centrality dictionary. Pay attention to the docstring (denoted on both sides by three sets of double quotation marks) for clues as to what purpose each parameter in the function serves.
</div>

In [ ]:
# Write a function that retrieves the top most central nodes from each centrality dictionary
def get_k_centrality(cents, top, K):
    """Takes in a list of centrality dictionaries and returns a list of 
    lists of nodes, representing either the top K or bottom K nodes as 
    measured by the centrality measure in question
    
    Args:
        cents (list): A list of centrality dictionaries.
        top (bool): If true, returns the nodes with the highest 
            centrality. If False, returns lowest.
        K (int): Number of nodes to retrieve from each dictionary.
        
    Returns:
        list: A list of lists of nodes.
    
    """
    
    # Initialize an empty list for storing the node subset lists
    final_list = []
    
    # iterate over the list of centralities passed to the function
    ▰▰1▰▰ ▰▰2▰▰ ▰▰3▰▰ ▰▰4▰▰:
        
        # Retrieve the top K nodes, as measured by the given centrality, and store them in a list
        # Note that the following list comprehension follows a similar pattern as the ones we used last week
        cent_list = [e for e in sorted(centrality, key=lambda x: centrality[x], reverse=▰▰5▰▰)[0:▰▰6▰▰]]

        # Add the resulting list of tuples to the overall list
        final_list.append(cent_list)
        
    # Return the list of lists
    return final_list

## Problem 4:
<div class="alert alert-block alert-info">  
Now that our function is in place, we can take advantage of its ability to take in multiple centrality dictionaries at a time. The plan here is to get the top 10 nodes and bottom 10 nodes from each of the four centrality dictionaries, for a total of 8 iterations. We could do all 8 right off the bat, but unpacking the resulting lists all on one line is a messy affair; instead, we'll split the task into two discrete steps! 
</div>
<div class="alert alert-block alert-success">
Use the 'get_k_centrality' function you completed in the previous question to get the top 10 nodes from each centrality dictionary. Using the printed top 10 lists as the basis for your answer, submit the ID of the node that appears in the first position (index 0) of three of the four centrality lists. 
</div>

In [ ]:
# Pass a list of each centrality dictionary to our function and unpack the returned value
# **Make sure that the order of the dictionaries matches the order of the unpacking variables**
top_shortest, top_current, top_degree, top_eigen = get_k_centrality(
    [▰▰1▰▰, 
     ▰▰2▰▰, 
     ▰▰3▰▰, 
     ▰▰4▰▰],
    top=▰▰5▰▰,
    K=▰▰6▰▰
)


print(top_shortest)
print(top_current)
print(top_degree)
print(top_eigen)

## Problem 5:
<div class="alert alert-block alert-info">  
Now for the bottom 10.
</div>
<div class="alert alert-block alert-success">
Use your 'get_k_centrality' function to get the bottom 10 nodes from each centrality dictionary. Using the printed bottom 10 lists as the basis for your answer, submit the ID of the node that appears in the first position (index 0) of two of the four centrality lists. 
</div>

In [ ]:
# Pass a list of each centrality dictionary to our function and unpack the returned value
# **Make sure that the order of the dictionaries matches the order of the unpacking variables**
bottom_shortest, bottom_current, bottom_degree, bottom_eigen = get_k_centrality(
    [▰▰1▰▰, 
     ▰▰2▰▰, 
     ▰▰3▰▰, 
     ▰▰4▰▰],
    top=▰▰5▰▰,
    K=▰▰6▰▰
)


print(bottom_shortest)
print(bottom_current)
print(bottom_degree)
print(bottom_eigen)

## Problem 6:
<div class="alert alert-block alert-info">  
Now we need a function that will allow us to put each infection set through its paces (using <code>multi_runs</code>). We'll approach this task in a similar manner as we did the previous function we built. Rather than tasking you with filling in many blanks, it's up to you to fill in just a single blank this time. The blank is for a built-in function that we'll use to randomly select 2 nodes from each list of nodes we pass to our function. The catch is that this built-in function isn't covered in the textbook or any of the other course material! It's up to you to examine the Python's official online documentation for the built-in <code>random</code> module and find the one you need.<br><br>
Even though there won't be any other explicit questions to answer about the function we're building in this question, it's still a good idea to work through it line-by-line and make sure you understand what's happening at each step. We've held back on inserting explanatory comments; see if you can figure out each part of the code for yourself!
</div>
<div class="alert alert-block alert-success">
Examine the documentation for Python's built-in <code>random</code> module to find a function that randomly samples a number of objects from a list (or other iterable). Use this function to retrieve, at random, two items from each infect_set list generated as part of the function below. Submit the name of the function you used (and only the name) exactly as you used it in your code (you should not need to include any punctuation or special symbols).
</div>

In [ ]:
def multi_run_infect_sets(sets, sir, runs, iterations):
    
    trend_list = []
    
    for infect_set in sets:
        
        # Create a list of 'infection sets', where each entry in the list is a list of two 
        # nodes randomly selected from those present in the 'infect_set' list
        setlist = [random.▰▰1▰▰(infect_set, k=2) for x in range(runs)]

        trends = multi_runs(
            sir, 
            execution_number=runs, 
            iteration_number=iterations, 
            infection_sets=setlist, 
            nprocesses=4)
        
        trend_list.append(trends)
        
    return trend_list
    

## Problem 7:
<div class="alert alert-block alert-info">  
Now, we're going to initialize the SIR model and put it through its paces! Using the same custom function we used in the textbook, we'll create a SIR model based on our Watts-Strogatz Graph and with <code>beta</code> and <code>gamma</code> values of 0.1. You might notice that we've set the <code>fraction_infected</code> parameter to 1, which means that -- by default -- all of the nodes will start infected. Normally, such initial conditions wouldn't be very interesting from a network diffusion perspective, but in our case, the initial infection parameter is meaningless; we're going to be overriding the initially-infected nodes using the lists we prepared earlier.
</div>
<div class="alert alert-block alert-success">
After instantiating the SIR model, use the <code>multi_run_infect_sets</code> function to produce trend results for each of the 8 node lists (4 each for top and bottom). Each model should be run 500 times, and each run should consist of 200 iterations.
</div>

In [ ]:
%%capture
sir_model_instance = sir_model(▰▰1▰▰, beta=▰▰2▰▰, gamma=▰▰3▰▰, fraction_infected=1)

top_shortest_trends, top_current_trends, top_degree_trends, top_eigen_trends = multi_run_infect_sets(
    sets = [top_shortest, 
     top_current, 
     top_degree, 
     top_eigen], 
    sir = ▰▰4▰▰,
    runs = ▰▰5▰▰,
    iterations = ▰▰6▰▰
)

bottom_shortest_trends, bottom_current_trends, bottom_degree_trends, bottom_eigen_trends = multi_run_infect_sets(
    [bottom_shortest, 
     bottom_current, 
     bottom_degree, 
     bottom_eigen], 
    ▰▰7▰▰,
    ▰▰8▰▰,
    ▰▰9▰▰
)

## Problem 8:
<div class="alert alert-block alert-info">  
With all the preiminaries in place, we can move onto the final steps: visualization and interpretation. In this question, we're going to ask you to interpret the SIR trend visualizations produced by nodes from each of the top-ten centrality lists. Pay attention to the differences between them; do any of them <b>significantly</b> stand out from the others? If you're having trouble telling the difference between any of them, then they're probably not significantly different. If, conversely, one or two of the visualizations show a contagion spreading much more rapidly, then those centralities are likely more impactful than the others. 
</div>
<div class="alert alert-block alert-success">
Select the statement that most closely matches your interpretation of the four visualizations below.
<br><br>
Note: the conclusions you draw for the purposes of this question should not be interpreted as a general truth about the relationship between diffusion dynamics and centrality in general. Your observations are specific to this network, are highly influenced by uncontrollable randomness, and should be interpreted as such. As a result of the randomness inherent to these models, it is possible that your interpretation will be correct, but will be graded as incorrect. If you feel that your interpretation was erroneously graded, please email screenshots of your visualizations from this question to Pierson. 
</div>

In [ ]:
top_trend_title_list = [
    (top_shortest_trends, "Top 5 Shortest-Path Betweenness Centrality"),
    (top_current_trends, "Top 5 Current Flow Betweenness Centrality"),
    (top_degree_trends, "Top 5 Degree Centrality"),
    (top_eigen_trends,"Top 5 Eigenvector Centrality"),
]

for trend, title in top_trend_title_list:
    print(title)
    visualize_trends(trend, network=G, return_data=False)

## Problem 9:
<div class="alert alert-block alert-info">  
Now, we'll do the same, but for the bottom ten nodes from each centrality measure. Since we're dealing with the bottom ten nodes here, 'more impactful' now corresponds with a significantly slower or less effective spread of the contagion.
</div>
<div class="alert alert-block alert-success">
Select the statement that most closely matches your interpretation of the four visualizations below.
<br><br>
Note: the conclusions you draw for the purposes of this question should not be interpreted as a general truth about the relationship between diffusion dynamics and centrality in general. Your observations are specific to this network, are highly influenced by uncontrollable randomness, and should be interpreted as such. As a result of the randomness inherent to these models, it is possible that your interpretation will be correct, but will be graded as incorrect. If you feel that your interpretation was erroneously graded, please email screenshots of your visualizations from this question to Pierson. 
</div>

In [ ]:
bottom_trend_title_list = [
    (bottom_shortest_trends,"Bottom 5 Shortest-Path Betweenness Centrality"),
    (bottom_current_trends,"Bottom 5 Current Flow Betweenness Centrality"),
    (bottom_degree_trends,"Bottom 5 Degree Centrality"),
    (bottom_eigen_trends,"Bottom 5 Eigenvector Centrality"),
]

for trend, title in bottom_trend_title_list:
    print(title)
    visualize_trends(trend, network=G, return_data=False)